In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import h5py

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
from matplotlib.pyplot import plot,savefig
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")
from read_count import read_data

import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from sklearn import metrics

In [2]:
col = ["#E64B35CC", "#0072B5CC", "#00A087CC", "#3C5488CC", "#F39B7FCC", "#F7DC05FF", "#FD7446E5",
       "#8491B4CC", "#7E6148CC", "#B09C85CC", "#E18727CC", "#FFDC91E5", "#6A6599E5", "#9467BDB2",
       "#FFFFFFFF", "#0000FFFF", "#FF0000FF", "#00FF00FF", "#000033FF", "#FF00B6FF", "#005300FF", "#FFD300FF",
       "#009FFFFF", "#9A4D42FF", "#00FFBEFF", "#783FC1FF", "#1F9698FF", "#FFACFDFF", "#B1CC71FF", "#F1085CFF",
       "#FE8F42FF", "#DD00FFFF", "#201A01FF", "#720055FF", "#766C95FF", "#02AD24FF", "#C8FF00FF", "#886C00FF",
       "#FFB79FFF", "#858567FF", "#A10300FF", "#14F9FFFF", "#00479EFF", "#DC5E93FF", "#93D4FFFF", "#004CFFFF"]

In [3]:
data_mat = h5py.File('dataset/Human_p.h5')
y_true_human = np.array(data_mat['Y'], dtype='int')
data_mat.close()

data_mat = h5py.File('dataset/Human_.h5')
y_true_pbmc = np.array(data_mat['Y'], dtype='int')
data_mat.close()

data_mat = h5py.File('dataset/Human_k.h5')
y_true_kidney = np.array(data_mat['Y'], dtype='int')
data_mat.close()

mat, obs, var, uns = read_data('dataset/Mouse_E.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_klein = np.unique(cell_name, return_inverse=True)

mat, obs, var, uns = read_data('dataset/Mouse_h.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_chen_1 = np.unique(cell_name, return_inverse=True)

adata = sc.AnnData(x)
adata.obs['celltype'] = y_true_chen_1
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)
y_true_chen_2 = np.array(adata.obs['celltype']).squeeze()

mat, obs, var, uns = read_data('dataset/Mouse_k.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_adam = np.unique(cell_name, return_inverse=True)

mat, obs, var, uns = read_data('dataset/Turtle_b.h5', sparsify=False, skip_exprs=False)
x = np.array(mat.toarray())
cell_name = np.array(obs["cell_type1"])
cell_type, y_true_turtle = np.unique(cell_name, return_inverse=True)

In [4]:
adam = np.load('results/scace_init/scace_init_adam.npz')
chen = np.load('results/scace_init/scace_init_chen.npz')
human = np.load('results/scace_init/scace_init_human.npz')
kidney = np.load('results/scace_init/scace_init_kidney.npz')
klein = np.load('results/scace_init/scace_init_klein.npz')
pbmc = np.load('results/scace_init/scace_init_pbmc.npz')
turtle = np.load('results/scace_init/scace_init_turtle.npz')

# UMAP

In [5]:
fig = plt.figure(figsize=(22, 5))
sub_figs = fig.subfigures(2, 1)
axs = []

for i, sub_fig in enumerate(sub_figs):
    axs.append(sub_fig.subplots(1, 7))
    
axs = np.array(axs)

In [6]:
def plot_cluster(df, n, data_name, y_true, by, ax):
    
    """
        by: 'pred' or 'true'. If by == 'pred', colored by cluster labels, else colored by true cell types.
        n: n-th dataset in [Mouse kidney, Mouse hypothalamus, Human pancreas, Human kidney, Mouse ES, Human PBMC, Turtle brain]
    """
    
    umap = umap_all[n]
    y_pred = df['Clusters']
    
    K = len(np.unique(y_pred))
    
    y_pred = np.asarray(y_pred, dtype='int').squeeze()
    
    if by == 'pred':
        ari = np.round(metrics.adjusted_rand_score(y_pred, y_true), 2)
        nmi = np.round(metrics.normalized_mutual_info_score(y_pred, y_true), 2)
        print('Dataset: {}, ARI={}, NMI={}, k={}'.format(data_name, ari, nmi, K))
        
    adata = sc.AnnData(pd.DataFrame(np.random.rand(len(y_pred), 1)))
    adata.obs['pred'] = y_pred
    adata.obs['pred'] = adata.obs['pred'].astype(int).astype('category')
    adata.obs['true'] = y_true
    adata.obs['true'] = adata.obs['true'].astype(int).astype('category')

    adata.obsm['X_umap'] = umap

    if by == 'pred':
        sc.pl.umap(adata, color=['pred'], ax=ax, show=False, legend_loc='None', size=8)
        ax.set_title('K={} ARI={}'.format(K, ari), fontsize=17, family='Arial')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        
    else:
        sc.pl.umap(adata, color=['true'], ax=ax, show=False, legend_loc='None', size=8, palette=col)
#         ax.set_title('K={}'.format(len(np.unique(y_true))), fontsize=17, family='Arial')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks([])

In [7]:
datasets = [adam, chen, human, kidney, klein, pbmc, turtle]

In [8]:
embedding = []
for data in datasets:
    embedding.append(data['Embedding'])
    
umap_all = []
for i in range(len(embedding)):
    print(i)
    adata = sc.AnnData(embedding[i])
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, random_state=0)
    umap_all.append(np.array(adata.obsm['X_umap']))

0
1
2
3
4
5
6


In [7]:
umap_all = np.load("umap/umap_scace_changer.npz")['UMAP']
# np.savez("umap/umap_scace_changer.npz", UMAP=umap_all)

In [8]:
plot_cluster(human, 2, 'Human pancreas', y_true_human, 'pred', axs[0][0])
plot_cluster(human, 2, 'Human pancreas', y_true_human, 'true', axs[1][0])

plot_cluster(pbmc, 5, 'Human PBMC', y_true_pbmc, 'pred', axs[0][1])
plot_cluster(pbmc, 5, 'Human PBMC', y_true_pbmc, 'true', axs[1][1])

plot_cluster(kidney, 3, 'Human kidney', y_true_kidney, 'pred', axs[0][2])
plot_cluster(kidney, 3, 'Human kidney', y_true_kidney, 'true', axs[1][2])

plot_cluster(klein, 4, 'Mouse ES', y_true_klein, 'pred', axs[0][3])
plot_cluster(klein, 4, 'Mouse ES', y_true_klein, 'true', axs[1][3])

plot_cluster(chen, 1, 'Mouse hypothalamus', y_true_chen, 'pred', axs[0][4])
plot_cluster(chen, 1, 'Mouse hypothalamus', y_true_chen, 'true', axs[1][4])

plot_cluster(adam, 0, 'Mouse kidney', y_true_adam, 'pred', axs[0][5])
plot_cluster(adam, 0, 'Mouse kidney', y_true_adam, 'true', axs[1][5])

plot_cluster(turtle, 6, 'Turtle brain', y_true_turtle, 'pred', axs[0][6])
plot_cluster(turtle, 6, 'Turtle brain', y_true_turtle, 'true', axs[1][6])

Dataset: Human pancreas, ARI=0.56, NMI=0.78, k=14
Dataset: Human PBMC, ARI=0.77, NMI=0.79, k=8
Dataset: Human kidney, ARI=0.62, NMI=0.75, k=11
Dataset: Mouse ES, ARI=0.98, NMI=0.96, k=4
Dataset: Mouse hypothalamus, ARI=0.33, NMI=0.71, k=46
Dataset: Human pancreas, ARI=0.91, NMI=0.89, k=8
Dataset: Turtle brain, ARI=0.6, NMI=0.78, k=14


In [9]:
fig

<Figure size 2200x500 with 14 Axes>

In [10]:
plt.savefig('Figures/FigureS4B.svg', dpi=300, format='svg', bbox_inches='tight')

# Metric

In [11]:
adam_ari_init = np.round(metrics.adjusted_rand_score(adam['Clusters'], y_true_adam), 2)
adam_nmi_init = np.round(metrics.normalized_mutual_info_score(adam['Clusters'], y_true_adam), 2)

chen_ari_init = np.round(metrics.adjusted_rand_score(chen['Clusters'], y_true_chen), 2)
chen_nmi_init = np.round(metrics.normalized_mutual_info_score(chen['Clusters'], y_true_chen), 2)

human_ari_init = np.round(metrics.adjusted_rand_score(human['Clusters'], y_true_human), 2)
human_nmi_init = np.round(metrics.normalized_mutual_info_score(human['Clusters'], y_true_human), 2)

kidney_ari_init = np.round(metrics.adjusted_rand_score(kidney['Clusters'], y_true_kidney), 2)
kidney_nmi_init = np.round(metrics.normalized_mutual_info_score(kidney['Clusters'], y_true_kidney), 2)

klein_ari_init = np.round(metrics.adjusted_rand_score(klein['Clusters'], y_true_klein), 2)
klein_nmi_init = np.round(metrics.normalized_mutual_info_score(klein['Clusters'], y_true_klein), 2)

pbmc_ari_init = np.round(metrics.adjusted_rand_score(pbmc['Clusters'], y_true_pbmc), 2)
pbmc_nmi_init = np.round(metrics.normalized_mutual_info_score(pbmc['Clusters'], y_true_pbmc), 2)

turtle_ari_init = np.round(metrics.adjusted_rand_score(turtle['Clusters'], y_true_turtle), 2)
turtle_nmi_init = np.round(metrics.normalized_mutual_info_score(turtle['Clusters'], y_true_turtle), 2)

In [12]:
adam = np.load('results/default/Adam/scAce_wo_sample.npz')
chen = np.load('results/default/Chen/scAce_wo_sample.npz')
human = np.load('results/default/Human/scAce_wo_sample.npz')
kidney = np.load('results/default/Kidney/scAce_wo_sample.npz')
klein = np.load('results/default/Klein/scAce_wo_sample.npz')
pbmc = np.load('results/default/PBMC/scAce_wo_sample.npz')
turtle = np.load('results/default/Turtle/scAce_wo_sample.npz')

In [13]:
adam_ari_last = adam['ARI']
adam_nmi_last = adam['NMI']

chen_ari_last = chen['ARI']
chen_nmi_last = chen['NMI']

human_ari_last = human['ARI']
human_nmi_last = human['NMI']

kidney_ari_last = kidney['ARI']
kidney_nmi_last = kidney['NMI']

klein_ari_last = klein['ARI']
klein_nmi_last = klein['NMI']

pbmc_ari_last = pbmc['ARI']
pbmc_nmi_last = pbmc['NMI']

turtle_ari_last = turtle['ARI']
turtle_nmi_last = turtle['NMI']

In [14]:
init_ari = [human_ari_init, pbmc_ari_init, kidney_ari_init, 
            klein_ari_init, chen_ari_init, adam_ari_init, turtle_ari_init]
init_nmi = [human_nmi_init, pbmc_nmi_init, kidney_nmi_init, 
            klein_nmi_init, chen_nmi_init, adam_nmi_init, turtle_nmi_init]
last_ari = [human_ari_last, pbmc_ari_last, kidney_ari_last, 
             klein_ari_last, chen_ari_last, adam_ari_last, turtle_ari_last]
last_nmi = [human_nmi_last, pbmc_nmi_last, kidney_nmi_last, 
             klein_nmi_last, chen_nmi_last, adam_nmi_last, turtle_nmi_last]

In [15]:
def plot_metric_merge(metric, ax):
    
    """
        metric: 'ARI' or 'NMI'
    """
    
    ax = plt.subplot(ax)
    
    plt.ylim(0, 1)
    
    bar_width = 0.35
    index = np.arange(7)

    if metric == 'ARI':
        plt.bar(index, init_ari, bar_width, alpha=1, color='#FB8930')
        plt.bar(index+bar_width, last_ari, bar_width, alpha=1, color='#5C89A0')
        
    else:
        plt.bar(index, init_nmi, bar_width, alpha=1, color='#FB8930')
        plt.bar(index+bar_width, last_nmi, bar_width, alpha=1, color='#5C89A0')
        
    x_labels = ['Human pancreas', 'Human PBMC', 'Human kidney', 'Mouse ES', 
                'Mouse hypothalamus', 'Mouse kidney', 'Turtle brain']
    plt.xticks(index, x_labels, fontsize=13, rotation=40, family='Arial', ha='right')
    plt.ylabel(metric, fontsize=17, family='Arial')

In [16]:
plt.figure(figsize=(15, 4))
plot_metric_merge('ARI', 121)
plot_metric_merge('NMI', 122)

In [17]:
plt.savefig('Figures/FigureS4A.svg', dpi=300, format='svg', bbox_inches='tight')